<a href="https://colab.research.google.com/github/fatenchtioui/Projet/blob/main/GAN%2BCancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from google.colab import drive

drive.mount('/content/drive')
dataset_path = '/content/drive/MyDrive/Colab Notebooks/train_cancer'

# Définir la taille du lot (batch_size)
batch_size = 64

# Charger vos données existantes (assurez-vous qu'elles sont correctement prétraitées)
X_train_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
y_train = X_train_gen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

# Obtenir un lot d'images réelles
batch_images, _ = next(y_train)

# Normaliser les données entre -1 et 1 (important pour GANs)
X_train = (batch_images.astype(np.float32) - 127.5) / 127.5

# ... (le reste de votre code)
import os

def save_generated_images(epoch, generator, examples=10, dim=(1, 10), figsize=(10, 1), save_path='/content/drive/MyDrive/Colab Notebooks/generated_images/'):
    noise = np.random.normal(0, 1, (examples, latent_dim))
    generated_images = generator.predict(noise)

    generated_images = generated_images * 0.5 + 0.5  # Rééchelonner les valeurs entre 0 et 1

    # Créer le répertoire s'il n'existe pas
    os.makedirs(save_path, exist_ok=True)

    fig, axs = plt.subplots(dim[0], dim[1], figsize=figsize)
    for i in range(dim[0] * dim[1]):
        axs[i].imshow(generated_images[i], interpolation='nearest', cmap='gray_r')
        axs[i].axis('off')

    plt.tight_layout()
    plt.savefig(f'{save_path}gan_generated_image_epoch_{epoch}.png')
    plt.close()

# ... (le reste de votre code)


# Fonction pour créer le générateur
def build_generator(latent_dim):
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, input_dim=latent_dim, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(np.prod(X_train.shape[1:]), activation='tanh'))
    model.add(layers.Reshape(X_train.shape[1:]))
    return model

# Fonction pour créer le discriminateur
def build_discriminator(img_shape):
    model = tf.keras.Sequential()
    model.add(layers.Flatten(input_shape=img_shape))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

# Fonction pour créer le GAN (combine le générateur et le discriminateur)
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = tf.keras.Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

# Paramètres
latent_dim = 100
img_shape = X_train.shape[1:]

# Construire et compiler le modèle du générateur
generator = build_generator(latent_dim)
generator.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0002, 0.5))

# Construire et compiler le modèle du discriminateur
discriminator = build_discriminator(img_shape)
discriminator.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0002, 0.5), metrics=['accuracy'])

# Construire et compiler le modèle GAN
discriminator.trainable = False
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0002, 0.5))

# Entraînement du GAN
epochs = 30000
batch_size = 64
half_batch = batch_size // 2

for epoch in range(epochs):
    # Entraînement du discriminateur
    idx = np.random.randint(0, X_train.shape[0], half_batch)
    imgs = X_train[idx]
    noise = np.random.normal(0, 1, (half_batch, latent_dim))
    gen_imgs = generator.predict(noise)

    d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
    d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Entraînement du générateur
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    valid_labels = np.array([1] * batch_size)
    g_loss = gan.train_on_batch(noise, valid_labels)

    # Affichage du progrès
    print(f"{epoch}/{epochs} [D loss: {d_loss[0]} | accuracy: {100 * d_loss[1]}] [G loss: {g_loss}]")

    # Sauvegarde des images générées à intervalles réguliers
    if epoch % 1000 == 0:
        save_generated_images(epoch, generator)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 224 images belonging to 2 classes.
1/1 [==============================] - 0s 415ms/step


0/30000 [D loss: 1.288327157497406 | accuracy: 50.0] [G loss: 1.0847232341766357]


1/1 [==============================] - 0s 423ms/step
1/30000 [D loss: 0.34854602813720703 | accuracy: 68.75] [G loss: 1.0833940505981445]
1/1 [==============================] - 0s 298ms/step
2/30000 [D loss: 0.17039397358894348 | accuracy: 100.0] [G loss: 1.1798856258392334]
1/1 [==============================] - 0s 474ms/step
3/30000 [D loss: 0.11905581504106522 | accuracy: 100.0] [G loss: 1.2617698907852173]
1/1 [==============================] - 0s 297ms/step
4/30000 [D loss: 0.12966755032539368 | accuracy: 100.0] [G loss: 1.1539572477340698]
1/1 [==============================] - 0s 436ms/step
5/30000 [D loss: 0.14309126138687447 | accuracy: 100.0] [G loss: 1.4278515577316284]
1/1 [==============================] - 0s 303ms/step
6/30000 [D loss: 0.23187259258702397 | accuracy: 100.0] [G loss: 1.3331689834594727]
1/1 [==============================] - 0s 454ms/step
7/30000 [D loss: 0.12764625251293182 | accuracy: 100.0] [G loss: 1.2807482481002808]
1/1 [=============================